In [1]:
# Install required libraries
!pip install -q transformers sentence-transformers rouge-score nltk bert_score

# Imports
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer, util
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from bert_score import score as bertscore
import nltk
from tqdm import tqdm

# Download NLTK tokenizer
nltk.download('punkt')

# Load dataset from Kaggle input
df = pd.read_csv("/kaggle/input/healthcare-emotion/emotion-emotion_69k.csv")  
df = df[['Situation', 'labels']].dropna().drop_duplicates().sample(30, random_state=42)
df.columns = ['question', 'true_answer']

# Load BioMedLM model
model_name = "stanford-crfm/biomedlm"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.eval()
torch.no_grad()

# Generate answers with BioMedLM
def ask_biomedlm(question, max_new_tokens=60):
    inputs = tokenizer(question, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=inputs["input_ids"].shape[1] + max_new_tokens)
    return tokenizer.decode(outputs[0], skip_special_tokens=True).replace(question, '').strip()

tqdm.pandas()
df['biomedlm_answer'] = df['question'].progress_apply(ask_biomedlm)

# Load Sentence-BERT model
sbert = SentenceTransformer('all-MiniLM-L6-v2')

# Compute Cosine Similarity
def cosine_similarity(a, b):
    emb1 = sbert.encode(a, convert_to_tensor=True)
    emb2 = sbert.encode(b, convert_to_tensor=True)
    return util.cos_sim(emb1, emb2).item()

df['cosine'] = df.apply(lambda row: cosine_similarity(row['true_answer'], row['biomedlm_answer']), axis=1)

# Compute ROUGE-L
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

def compute_rouge(a, b):
    return scorer.score(a, b)['rougeL'].fmeasure

df['rougeL'] = df.apply(lambda row: compute_rouge(row['true_answer'], row['biomedlm_answer']), axis=1)

# Compute BLEU
def compute_bleu(reference, hypothesis):
    ref_tokens = nltk.word_tokenize(reference.lower())
    hyp_tokens = nltk.word_tokenize(hypothesis.lower())
    smoothing = SmoothingFunction().method1
    return sentence_bleu([ref_tokens], hyp_tokens, smoothing_function=smoothing)

df['bleu'] = df.apply(lambda row: compute_bleu(row['true_answer'], row['biomedlm_answer']), axis=1)

# Compute BERTScore
P, R, F1 = bertscore(
    df['biomedlm_answer'].tolist(),
    df['true_answer'].tolist(),
    lang='en',
    rescale_with_baseline=True
)
df['bertscore_f1'] = F1

# Print evaluation scores
print("Average Cosine Similarity:", df['cosine'].mean())
print("Average ROUGE-L:", df['rougeL'].mean())
print("Average BLEU:", df['bleu'].mean())
print("Average BERTScore-F1:", df['bertscore_f1'].mean())

# Save results
df.to_csv("biomedlm_qa_results_with_bleu_bertscore.csv", index=False)
df[['question', 'true_answer', 'biomedlm_answer', 'cosine', 'rougeL', 'bleu', 'bertscore_f1']].head(10)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.9 MB/s eta 0:00:

2025-07-27 14:54:20.607331: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753628060.789926      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753628060.848456      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


tokenizer_config.json:   0%|          | 0.00/267 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Average Cosine Similarity: 0.07993002686028679
Average ROUGE-L: 0.04614606486856325
Average BLEU: 0.004443256575764966
Average BERTScore-F1: -1.0426409


,question,true_answer,biomedlm_answer,cosine,rougeL,bleu,bertscore_f1
49098,my sister is having her first baby,Oh fun! A little girl will be a great addition...,", and the other half of the family members, an...",0.093562,0.064516,0.008361,0.006680
60057,I woke up this morning and saw a scratch on my...,I hope so too. Its really annoying when that h...,I think it is important to be able to do it. I...,0.173824,0.035714,0.004635,-0.053166
36989,Getting my things ready for florida,The family and I are headed to florida in a co...,**,0.093602,0.000000,0.000000,-0.268575
21737,I was a bit shocked that someone was so nice t...,I'm off that train so they aren't coming back ...,I think it's a bit of a problem. I think it's ...,0.092706,0.028169,0.003616,-0.046193
61907,I injured my foot a couple of days ago (muscle...,"don't worry, with people like this karma hits ...",,0.104818,0.000000,0.000000,-4.925075
34677,Each month I am scared to look at my bank acco...,Why do you say that?,to be considered as a potential source of info...,0.086509,0.000000,0.000000,-0.134753
55676,When my boyfriend and I had been dating about ...,Well that's good. I'm glad so far it has worke...,I was not sure I was going to go to the hospit...,0.074471,0.061538,0.007367,-0.035080
50788,"Boy is it hard to be alone sometimes, this pas...",I know how you feel. What helped me was gettin...,I think that the authors have a good idea of w...,0.061033,0.166667,0.011876,0.192653
12507,I am nervous because I am having surgery tomorrow,is it a major or minor surgery?,", I will be able to do it. I am not sure how t...",0.059553,0.111111,0.004459,0.016527
51177,Water is not working in the house. I have been...,I have been waiting on the plumber all day and...,The authors declare no conflict of interest.\n...,0.041070,0.058824,0.008301,-0.272538


In [1]:
# Install required libraries
!pip install -q transformers sentence-transformers rouge-score nltk bert_score

# Imports
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer, util
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from bert_score import score as bertscore
import nltk
from tqdm import tqdm

# Download NLTK tokenizer
nltk.download('punkt')

# Load dataset from Kaggle input
df = pd.read_csv("/kaggle/input/mental/emotion-emotion_69k.csv")  
df = df[['Situation', 'labels']].dropna().drop_duplicates().sample(30, random_state=42)
df.columns = ['question', 'true_answer']

# Load MedAlpaca model
tokenizer = AutoTokenizer.from_pretrained("medalpaca/medalpaca-7b")
model = AutoModelForCausalLM.from_pretrained("medalpaca/medalpaca-7b", torch_dtype=torch.float16, device_map="auto")
model.eval()
torch.no_grad()

# Generate answers with MedAlpaca
def generate_answer(question, max_new_tokens=100):
    inputs = tokenizer(question, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=inputs["input_ids"].shape[1] + max_new_tokens,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer.replace(question, '').strip()

tqdm.pandas()
df['medalpaca_answer'] = df['question'].progress_apply(generate_answer)

# Load Sentence-BERT model
sbert = SentenceTransformer('all-MiniLM-L6-v2')

# Compute Cosine Similarity
def cosine_similarity(a, b):
    emb1 = sbert.encode(a, convert_to_tensor=True)
    emb2 = sbert.encode(b, convert_to_tensor=True)
    return util.cos_sim(emb1, emb2).item()

df['cosine'] = df.apply(lambda row: cosine_similarity(row['true_answer'], row['medalpaca_answer']), axis=1)

# Compute ROUGE-L
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

def compute_rouge(a, b):
    return scorer.score(a, b)['rougeL'].fmeasure

df['rougeL'] = df.apply(lambda row: compute_rouge(row['true_answer'], row['medalpaca_answer']), axis=1)

# Compute BLEU
def compute_bleu(reference, hypothesis):
    ref_tokens = nltk.word_tokenize(reference.lower())
    hyp_tokens = nltk.word_tokenize(hypothesis.lower())
    smoothing = SmoothingFunction().method1
    return sentence_bleu([ref_tokens], hyp_tokens, smoothing_function=smoothing)

df['bleu'] = df.apply(lambda row: compute_bleu(row['true_answer'], row['medalpaca_answer']), axis=1)

# Compute BERTScore (filtered with rescaling)
valid_df = df[(df['medalpaca_answer'].str.strip() != '') & (df['true_answer'].str.strip() != '')]
P, R, F1 = bertscore(
    valid_df['medalpaca_answer'].tolist(),
    valid_df['true_answer'].tolist(),
    lang='en',
    rescale_with_baseline=True
)
df.loc[valid_df.index, 'bertscore_f1'] = F1.tolist()

# Print evaluation scores
print("Average Cosine Similarity:", df['cosine'].mean())
print("Average ROUGE-L:", df['rougeL'].mean())
print("Average BLEU:", df['bleu'].mean())
print("Average BERTScore-F1:", df['bertscore_f1'].mean())

# Save results
df.to_csv("medalpaca_qa_results_with_bleu_bertscore.csv", index=False)
df[['question', 'true_answer', 'medalpaca_answer', 'cosine', 'rougeL', 'bleu', 'bertscore_f1']].head(10)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 724.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 6.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 2.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.2 MB/s 

2025-07-27 17:39:30.610781: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753637970.811345      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753637970.874390      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


tokenizer_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Average Cosine Similarity: 0.19836801588535308
Average ROUGE-L: 0.07977471718462321
Average BLEU: 0.0075221327018880485
Average BERTScore-F1: -0.0161176186054945


,question,true_answer,medalpaca_answer,cosine,rougeL,bleu,bertscore_f1
49098,my sister is having her first baby,Oh fun! A little girl will be a great addition...,in August and she was so excited to have a boy...,0.422627,0.131868,0.005858,0.057731
60057,I woke up this morning and saw a scratch on my...,I hope so too. Its really annoying when that h...,"I am a single mother of three children, I am a...",-0.006505,0.076923,0.002945,0.021634
36989,Getting my things ready for florida,The family and I are headed to florida in a co...,and then getting ready for the wedding!\n✔️ Pa...,0.543381,0.120000,0.013948,0.026431
21737,I was a bit shocked that someone was so nice t...,I'm off that train so they aren't coming back ...,"Anyway, I am so excited to be going home for T...",0.149257,0.102564,0.007220,-0.025581
61907,I injured my foot a couple of days ago (muscle...,"don't worry, with people like this karma hits ...","I'm not crippled, but I can't run as fast as I...",0.167847,0.123077,0.005809,0.094833
34677,Each month I am scared to look at my bank acco...,Why do you say that?,for my bills and I have to borrow from Peter t...,-0.077851,0.000000,0.000000,-0.206115
55676,When my boyfriend and I had been dating about ...,Well that's good. I'm glad so far it has worke...,The apartment complex was run by a slumlord. T...,0.070849,0.068182,0.005488,-0.065807
50788,"Boy is it hard to be alone sometimes, this pas...",I know how you feel. What helped me was gettin...,"I went to the doctor, I took some medicine, I ...",0.181119,0.090090,0.005768,0.100056
12507,I am nervous because I am having surgery tomorrow,is it a major or minor surgery?,. The surgery will hopefully help my back and ...,0.389701,0.042105,0.002558,-0.014695
51177,Water is not working in the house. I have been...,I have been waiting on the plumber all day and...,The plumber said he would be here between 8 an...,0.568783,0.204082,0.046473,0.228326


In [1]:
# Install required libraries
!pip install -q transformers sentence-transformers rouge-score nltk bert_score

# Imports
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer, util
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from bert_score import score as bertscore
import nltk
from tqdm import tqdm

# Download NLTK tokenizer
nltk.download('punkt')

# Load dataset from Kaggle input
df = pd.read_csv("/kaggle/input/mental/emotion-emotion_69k.csv")  
df = df[['Situation', 'labels']].dropna().drop_duplicates().sample(30, random_state=42)
df.columns = ['question', 'true_answer']

# Load MedLLaMA2 model
tokenizer = AutoTokenizer.from_pretrained("llSourcell/medllama2_7b")
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model = AutoModelForCausalLM.from_pretrained("llSourcell/medllama2_7b")
model.resize_token_embeddings(len(tokenizer))
model = model.to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()
torch.no_grad()

# Generate answers with MedLLaMA2
def generate_answer(question, max_new_tokens=100):
    inputs = tokenizer(question, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=inputs["input_ids"].shape[1] + max_new_tokens,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
        pad_token_id=tokenizer.pad_token_id
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer.replace(question, '').strip()

tqdm.pandas()
df['medllama2_answer'] = df['question'].progress_apply(generate_answer)

# Load Sentence-BERT model
sbert = SentenceTransformer('all-MiniLM-L6-v2')

# Compute Cosine Similarity
def cosine_similarity(a, b):
    emb1 = sbert.encode(a, convert_to_tensor=True)
    emb2 = sbert.encode(b, convert_to_tensor=True)
    return util.cos_sim(emb1, emb2).item()

df['cosine'] = df.apply(lambda row: cosine_similarity(row['true_answer'], row['medllama2_answer']), axis=1)

# Compute ROUGE-L
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

def compute_rouge(a, b):
    return scorer.score(a, b)['rougeL'].fmeasure

df['rougeL'] = df.apply(lambda row: compute_rouge(row['true_answer'], row['medllama2_answer']), axis=1)

# Compute BLEU
def compute_bleu(reference, hypothesis):
    ref_tokens = nltk.word_tokenize(reference.lower())
    hyp_tokens = nltk.word_tokenize(hypothesis.lower())
    smoothing = SmoothingFunction().method1
    return sentence_bleu([ref_tokens], hyp_tokens, smoothing_function=smoothing)

df['bleu'] = df.apply(lambda row: compute_bleu(row['true_answer'], row['medllama2_answer']), axis=1)

# Compute BERTScore (filtered with rescaling)
valid_df = df[(df['medllama2_answer'].str.strip() != '') & (df['true_answer'].str.strip() != '')]
P, R, F1 = bertscore(
    valid_df['medllama2_answer'].tolist(),
    valid_df['true_answer'].tolist(),
    lang='en',
    rescale_with_baseline=True
)
df.loc[valid_df.index, 'bertscore_f1'] = F1.tolist()

# Print evaluation scores
print("Average Cosine Similarity:", df['cosine'].mean())
print("Average ROUGE-L:", df['rougeL'].mean())
print("Average BLEU:", df['bleu'].mean())
print("Average BERTScore-F1:", df['bertscore_f1'].mean())

# Save results
df.to_csv("medllama2_qa_results_with_bleu_bertscore.csv", index=False)
df[['question', 'true_answer', 'medllama2_answer', 'cosine', 'rougeL', 'bleu', 'bertscore_f1']].head(10)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.8 MB/s eta 0:00

2025-07-27 18:03:26.436890: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753639406.697545      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753639406.777010      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Average Cosine Similarity: 0.21874920365711054
Average ROUGE-L: 0.07752352280547284
Average BLEU: 0.005516076281162819
Average BERTScore-F1: 0.02490196693688631


,question,true_answer,medllama2_answer,cosine,rougeL,bleu,bertscore_f1
49098,my sister is having her first baby,Oh fun! A little girl will be a great addition...,. I’m so excited! I’m going to be an auntie! 😍...,0.443977,0.074074,0.005698,0.045094
60057,I woke up this morning and saw a scratch on my...,I hope so too. Its really annoying when that h...,What a bummer. I just washed it last week. I t...,0.177117,0.083333,0.006079,0.045175
36989,Getting my things ready for florida,The family and I are headed to florida in a co...,. I'm so excited!\nWe are planning a trip to f...,0.690341,0.141414,0.006898,0.136251
21737,I was a bit shocked that someone was so nice t...,I'm off that train so they aren't coming back ...,But it's important to remember that people can...,0.071981,0.080000,0.003218,0.026313
61907,I injured my foot a couple of days ago (muscle...,"don't worry, with people like this karma hits ...",It's really upsetting. What can I do? [/INST...,0.160316,0.043478,0.003639,0.009117
34677,Each month I am scared to look at my bank acco...,Why do you say that?,to cover all my expenses. I have to use my cre...,-0.037879,0.023529,0.002842,-0.158900
55676,When my boyfriend and I had been dating about ...,Well that's good. I'm glad so far it has worke...,We found a really affordable one and he was so...,0.043999,0.088235,0.007949,0.028772
50788,"Boy is it hard to be alone sometimes, this pas...",I know how you feel. What helped me was gettin...,"I hope you are doing well, I know it's not eas...",0.316930,0.141414,0.006582,0.116356
12507,I am nervous because I am having surgery tomorrow,is it a major or minor surgery?,. I have been feeling a little anxious and am ...,0.220640,0.075949,0.003263,-0.014699
51177,Water is not working in the house. I have been...,I have been waiting on the plumber all day and...,"It's so hot outside, and I'm feeling so uncomf...",0.028761,0.101695,0.005978,0.145245


In [1]:
# Install required libraries
!pip install -q transformers sentence-transformers rouge-score nltk bert_score

# Imports
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer, util
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from bert_score import score as bertscore
import nltk
from tqdm import tqdm

# Download NLTK tokenizer
nltk.download('punkt')

# Load dataset from Kaggle input
df = pd.read_csv("/kaggle/input/mental/emotion-emotion_69k.csv")  
df = df[['Situation', 'labels']].dropna().drop_duplicates().sample(30, random_state=42)
df.columns = ['question', 'true_answer']

# Load EleutherAI GPT-Neo 1.3B model
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
model.resize_token_embeddings(len(tokenizer))
model = model.to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()
torch.no_grad()

# Generate answers with GPT-Neo
def generate_answer(question, max_new_tokens=100):
    inputs = tokenizer(question, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=inputs["input_ids"].shape[1] + max_new_tokens,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
        pad_token_id=tokenizer.pad_token_id
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer.replace(question, '').strip()

tqdm.pandas()
df['gptneo_answer'] = df['question'].progress_apply(generate_answer)

# Load Sentence-BERT model
sbert = SentenceTransformer('all-MiniLM-L6-v2')

# Compute Cosine Similarity
def cosine_similarity(a, b):
    emb1 = sbert.encode(a, convert_to_tensor=True)
    emb2 = sbert.encode(b, convert_to_tensor=True)
    return util.cos_sim(emb1, emb2).item()

df['cosine'] = df.apply(lambda row: cosine_similarity(row['true_answer'], row['gptneo_answer']), axis=1)

# Compute ROUGE-L
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

def compute_rouge(a, b):
    return scorer.score(a, b)['rougeL'].fmeasure

df['rougeL'] = df.apply(lambda row: compute_rouge(row['true_answer'], row['gptneo_answer']), axis=1)

# Compute BLEU
def compute_bleu(reference, hypothesis):
    ref_tokens = nltk.word_tokenize(reference.lower())
    hyp_tokens = nltk.word_tokenize(hypothesis.lower())
    smoothing = SmoothingFunction().method1
    return sentence_bleu([ref_tokens], hyp_tokens, smoothing_function=smoothing)

df['bleu'] = df.apply(lambda row: compute_bleu(row['true_answer'], row['gptneo_answer']), axis=1)

# Compute BERTScore (filtered with rescaling)
valid_df = df[(df['gptneo_answer'].str.strip() != '') & (df['true_answer'].str.strip() != '')]
P, R, F1 = bertscore(
    valid_df['gptneo_answer'].tolist(),
    valid_df['true_answer'].tolist(),
    lang='en',
    rescale_with_baseline=True
)
df.loc[valid_df.index, 'bertscore_f1'] = F1.tolist()

# Print evaluation scores
print("Average Cosine Similarity:", df['cosine'].mean())
print("Average ROUGE-L:", df['rougeL'].mean())
print("Average BLEU:", df['bleu'].mean())
print("Average BERTScore-F1:", df['bertscore_f1'].mean())

# Save results
df.to_csv("gptneo_qa_results_with_bleu_bertscore.csv", index=False)
df[['question', 'true_answer', 'gptneo_answer', 'cosine', 'rougeL', 'bleu', 'bertscore_f1']].head(10)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.3 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.2 MB/s eta 0:00

2025-07-27 19:22:00.191749: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753644120.416736      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753644120.472542      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Average Cosine Similarity: 0.20127408172314365
Average ROUGE-L: 0.06942374379331945
Average BLEU: 0.005828728034745211
Average BERTScore-F1: -0.02375850873067975


,question,true_answer,gptneo_answer,cosine,rougeL,bleu,bertscore_f1
49098,my sister is having her first baby,Oh fun! A little girl will be a great addition...,", so we are all over the place with her. My mo...",0.215766,0.078431,0.006100,0.049996
60057,I woke up this morning and saw a scratch on my...,I hope so too. Its really annoying when that h...,I was just in the neighborhood and noticed the...,0.151123,0.057692,0.002884,-0.025463
36989,Getting my things ready for florida,The family and I are headed to florida in a co...,"I’m a newbie to the blogging world, and I am l...",0.197846,0.140000,0.005943,0.013264
21737,I was a bit shocked that someone was so nice t...,I'm off that train so they aren't coming back ...,I don't know what it is about people. I'm real...,0.000014,0.095238,0.005331,-0.001246
61907,I injured my foot a couple of days ago (muscle...,"don't worry, with people like this karma hits ...",I just want to say I'm sorry for the trouble I...,0.103327,0.021053,0.002908,-0.058393
34677,Each month I am scared to look at my bank acco...,Why do you say that?,to cover the bills and I end up having to pay ...,0.096939,0.040404,0.002147,-0.245999
55676,When my boyfriend and I had been dating about ...,Well that's good. I'm glad so far it has worke...,"It was a large apartment, and we were able to ...",0.010366,0.043011,0.004599,-0.073884
50788,"Boy is it hard to be alone sometimes, this pas...",I know how you feel. What helped me was gettin...,There were times when I felt like I was going ...,0.302039,0.076190,0.002906,-0.029781
12507,I am nervous because I am having surgery tomorrow,is it a major or minor surgery?,. I am supposed to have a CAT scan and it has ...,0.211086,0.042105,0.002661,-0.016747
51177,Water is not working in the house. I have been...,I have been waiting on the plumber all day and...,I am waiting for him to come out and fix the w...,0.594928,0.093458,0.005294,0.159520


In [1]:
# Install required libraries
!pip install -q transformers sentence-transformers rouge-score tqdm sacremoses nltk bert-score

# Imports
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer, util
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from bert_score import score as bertscore
import nltk
from tqdm import tqdm

# Download NLTK tokenizer
nltk.download('punkt')

# Load dataset from Kaggle input
df = pd.read_csv("/kaggle/input/mental/emotion-emotion_69k.csv")  
df = df[['Situation', 'labels']].dropna().drop_duplicates().sample(30, random_state=42)
df.columns = ['question', 'true_answer']

# Load BioGPT model
biogpt_name = "microsoft/BioGPT-Large"
tokenizer = AutoTokenizer.from_pretrained(biogpt_name)
model = AutoModelForCausalLM.from_pretrained(biogpt_name).to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()
torch.no_grad()

# Generate answers with BioGPT
def generate_answer(question, max_new_tokens=100):
    inputs = tokenizer(question, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=inputs["input_ids"].shape[1] + max_new_tokens,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer.replace(question, '').strip()

tqdm.pandas()
df['biogpt_answer'] = df['question'].progress_apply(generate_answer)

# Load Sentence-BERT model
sbert = SentenceTransformer('all-MiniLM-L6-v2')

# Compute Cosine Similarity
def cosine_similarity(a, b):
    emb1 = sbert.encode(a, convert_to_tensor=True)
    emb2 = sbert.encode(b, convert_to_tensor=True)
    return util.cos_sim(emb1, emb2).item()

df['cosine'] = df.apply(lambda row: cosine_similarity(row['true_answer'], row['biogpt_answer']), axis=1)

# Compute ROUGE-L
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

def compute_rouge(a, b):
    return scorer.score(a, b)['rougeL'].fmeasure

df['rougeL'] = df.apply(lambda row: compute_rouge(row['true_answer'], row['biogpt_answer']), axis=1)

# Compute BLEU
def compute_bleu(reference, hypothesis):
    ref_tokens = nltk.word_tokenize(reference.lower())
    hyp_tokens = nltk.word_tokenize(hypothesis.lower())
    smoothing = SmoothingFunction().method1
    return sentence_bleu([ref_tokens], hyp_tokens, smoothing_function=smoothing)

df['bleu'] = df.apply(lambda row: compute_bleu(row['true_answer'], row['biogpt_answer']), axis=1)

# Compute BERTScore (filtered with rescaling)
valid_df = df[(df['biogpt_answer'].str.strip() != '') & (df['true_answer'].str.strip() != '')]
P, R, F1 = bertscore(
    valid_df['biogpt_answer'].tolist(),
    valid_df['true_answer'].tolist(),
    lang='en',
    rescale_with_baseline=True
)
df.loc[valid_df.index, 'bertscore_f1'] = F1.tolist()

# Print evaluation scores
print("Average Cosine Similarity:", df['cosine'].mean())
print("Average ROUGE-L:", df['rougeL'].mean())
print("Average BLEU:", df['bleu'].mean())
print("Average BERTScore-F1:", df['bertscore_f1'].mean())

# Save results
df.to_csv("biogpt_qa_results_with_bleu_bertscore.csv", index=False)
df[['question', 'true_answer', 'biogpt_answer', 'cosine', 'rougeL', 'bleu', 'bertscore_f1']].head(10)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:010

2025-07-27 19:48:08.864891: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753645689.046908      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753645689.099086      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Average Cosine Similarity: 0.1285142518269519
Average ROUGE-L: 0.050639930504391736
Average BLEU: 0.004396355902487333
Average BERTScore-F1: -0.20551200010813772


,question,true_answer,biogpt_answer,cosine,rougeL,bleu,bertscore_f1
49098,my sister is having her first baby,Oh fun! A little girl will be a great addition...,. < / FREETEXT > < / TITLE > ▃,0.032967,0.000000,0.000000,-0.370707
60057,I woke up this morning and saw a scratch on my...,I hope so too. Its really annoying when that h...,I thought I was wandering about the corner of ...,0.048958,0.055556,0.007426,-0.096783
36989,Getting my things ready for florida,The family and I are headed to florida in a co...,. < / FREETEXT > < / TITLE > ▃,-0.042650,0.000000,0.004261,-0.402081
21737,I was a bit shocked that someone was so nice t...,I'm off that train so they aren't coming back ...,"I was really tired, and I was not so happy tha...",0.081115,0.040404,0.002719,-0.020801
61907,I injured my foot a couple of days ago (muscle...,"don't worry, with people like this karma hits ...",I injured my foot a couple of days ago (muscle...,0.088389,0.068966,0.004729,-0.101220
34677,Each month I am scared to look at my bank acco...,Why do you say that?,. I'm worried that I'm not getting paid proper...,0.107543,0.074074,0.005834,-0.222283
55676,When my boyfriend and I had been dating about ...,Well that's good. I'm glad so far it has worke...,The result was a 1 2-year-old girl who could g...,0.049771,0.025974,0.002765,-0.108254
50788,"Boy is it hard to be alone sometimes, this pas...",I know how you feel. What helped me was gettin...,"Sometimes I'm not alone, I'm not alone. I feel...",0.161743,0.083333,0.006369,-0.047869
12507,I am nervous because I am having surgery tomorrow,is it a major or minor surgery?,. < / FREETEXT > < / TITLE > ▃,0.018883,0.000000,0.000000,-0.396771
51177,Water is not working in the house. I have been...,I have been waiting on the plumber all day and...,My wife has said that I'm not willing to go to...,0.534550,0.157895,0.016276,0.044956
